In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [2,950 kB]
Hit:14 http://ppa.launchpad.net/graphics-dri

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [7]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3LI5TRP3YIDQL|          5|            0|          0|   N|                Y|
|R3LGC3EKEG84PX|          5|            0|          0|   N|                Y|
| R9PYL3OYH55QY|          5|            0|          1|   N|                Y|
|R3PWBAWUS4NT0Q|          3|            0|          0|   N|                Y|
|R15LYP3O51UU9E|          5|            0|          0|   N|                Y|
|R1AD7L0CC3DSRI|          5|            0|          0|   N|                Y|
|R32FE8Y45QV434|          5|            0|          0|   N|                Y|
|R3NM4MZ4XWL43Q|          5|            1|          2|   N|                Y|
|R3H4FXX6Q7I37D|          4|            0|          0|   N|                Y|
|R30L5PET7LFFDC|          5|            1|          1|   N|     

In [8]:
from pyspark.sql.functions import col #importing col function
total_votes_df = vine_df.filter(col("total_votes") >= 20)
total_votes_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N|
| RO8RAEGB66BKR|          4|           46|         46|   N|                N|
| RRFZ7QZTRJC59|          5|          292|        300|   N|                N|
| RFN4PNRUD1UW2|          4|           21|         22|   N|                N|
| RO7EBJEP7IHIX|          5|           26|         26|   N|                N|
|R1CVS4MK9RTNNP|          2|           11|         22|   N|     

In [31]:
from pyspark.sql.functions import concat,lit,round
helpful_total_df = total_votes_df.withColumn('helpful_total_ratio',round(col("helpful_votes") / col("total_votes")*100,0))
helpful_total_df= helpful_total_df.filter(col("helpful_total_ratio") >= 50)
helpful_total_df= helpful_total_df.withColumn('helpful_total_ratio',concat(col("helpful_total_ratio").cast("int"),lit('%')))
helpful_total_df.show(20)


+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_total_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y|                96%|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N|                96%|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|                95%|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y|                90%|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N|                96%|
| RO8RAEGB66BKR|          4|           46|         46|   N|                N|               100%|
| RRFZ7QZTRJC59|          5|          292|        300|   N|                N|                97%|
| RFN4PNRUD1UW2|    

In [34]:
review_paid_df=helpful_total_df.filter(col("vine") == 'Y')
review_paid_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_total_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
| R84VUCDBCI29U|          3|           18|         28|   Y|                N|                64%|
| R4V3ICFDTIDIF|          4|           20|         21|   Y|                N|                95%|
|R1JZ0JAPW83WFS|          4|           54|         58|   Y|                N|                93%|
|R1XH7EA97FAVP7|          3|           35|         44|   Y|                N|                80%|
|R1482JAU1ZR7QH|          4|           15|         22|   Y|                N|                68%|
|R1GGYGVTHP84DG|          4|           25|         27|   Y|                N|                93%|
| RXGU9DSKZJSP0|          3|           21|         22|   Y|                N|                95%|
+--------------+----

In [35]:
review_unpaid_df=helpful_total_df.filter(col("vine") == 'N')
review_unpaid_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_total_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y|                96%|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N|                96%|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|                95%|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y|                90%|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N|                96%|
| RO8RAEGB66BKR|          4|           46|         46|   N|                N|               100%|
| RRFZ7QZTRJC59|          5|          292|        300|   N|                N|                97%|
| RFN4PNRUD1UW2|    

In [60]:
number_of_reviews =vine_df.count()
five_star_reviews=vine_df.filter(col("star_rating") == 5).count()
paid_count=vine_df.filter((col("star_rating") == 5) & (col("vine")=="Y") ).count()
unpaid_count=vine_df.filter((col("star_rating") == 5) & (col("vine")=="N")).count()

In [73]:

print('Total Reviews: ' + f'{number_of_reviews:,}' )
print('Total 5 Star Reviews: ' + f'{five_star_reviews:,}')
print('Total Paid 5 Star Reviews: ' + f'{paid_count:,}')
print('Total Unpaid 5 Star Reviews: ' + f'{unpaid_count:,}')
print('Percent of Paid 5 Star Reviews: ' + f"{paid_count/five_star_reviews :.2%}" )
print('Percent of Unpaid 5 Star Reviews: ' + f"{unpaid_count/five_star_reviews :.2%}" )

Total Reviews: 4,751,577
Total 5 Star Reviews: 3,290,473
Total Paid 5 Star Reviews: 334
Total Unpaid 5 Star Reviews: 3,290,139
Percent of Paid 5 Star Reviews: 0.01%
Percent of Unpaid 5 Star Reviews: 99.99%


In [101]:
vine_count = vine_df.filter(col('vine') == 'Y').count()
non_vine_count = vine_df.filter(col('vine') == 'N').count()
print('Total Vine Reviews: '  + f'{vine_count:,}')
print('Total Non-Vine Reviews: '  + f'{non_vine_count:,}')

Total Vine Reviews: 1,933
Total Non-Vine Reviews: 4,749,607


In [96]:
print('| Total Reviews | 5 Star Reviews |  Paid 5 Star Reviews | Unpaid 5 Star Reviews | Percent of Paid | Percent of Unpaid | ')
print('-----------------------------------------------------------------------------------------------------------------------')
str1 = '|   ' + f'{number_of_reviews:,}' + '   |   ' + f'{five_star_reviews:,}' +'    |         ' + f'{paid_count:,}' + '          |        ' + f'{unpaid_count:,}' + '      |'
str2 ='      ' +  f'{paid_count/five_star_reviews :.2%}' + '      |      ' + f'{unpaid_count/five_star_reviews :.2%}' + '       |'
print(str1 + str2)

| Total Reviews | 5 Star Reviews |  Paid 5 Star Reviews | Unpaid 5 Star Reviews | Percent of Paid | Percent of Unpaid | 
-----------------------------------------------------------------------------------------------------------------------
|   4,751,577   |   3,290,473    |         334          |        3,290,139      |      0.01%      |      99.99%       |
